Valerie's Thoughts on Pagination
2023.08.22 VLB

API pagination is a crucial concept when dealing with large sets of data from APIs. It ensures that you receive data in manageable chunks rather than overwhelming your system. 
Let's explore 3 common approaches to handle paginated API endpoints using Python, specifically the `https://rickandmortyapi.com/api/character` API as an example.

1 and 2 are most commonly utlized


Brief (most recent runtime) analysis of execution times
Approach 1: 13.6s
Approach 2: 11.3s
Approach 1: 11.8s

so, maybe option 2 is best?

In [4]:
# **Approach 1: Using a While Loop**
# In this approach, we'll use a `while` loop to repeatedly make requests to the API and retrieve paginated data until there is no more data left.

import requests

def fetch_characters(url):
    all_characters = [] #initialize an empty list to store the deserialized response 

    while url:
        response = requests.get(url)
        data = response.json()
        all_characters.extend(data['results'])
        url = data['info']['next']  # Get URL of next page

    return all_characters

base_url = "https://rickandmortyapi.com/api/character"
all_characters = fetch_characters(base_url)

print(f"Total characters: {len(all_characters)}")

Total characters: 826


Explanation Approach 1:
1. We start by defining a function `fetch_characters` that takes the URL of the API endpoint as its parameter.
2. Inside the function, we initialize an empty list `all_characters` to store all the characters retrieved.
3. We use a `while` loop to repeatedly make API requests until there is no more `next` page URL available.
4. Within the loop, we send a GET request to the current URL and retrieve the JSON response.
5. We extend the `all_characters` list with the `results` from the response data, which contains characters for the current page.
6. We update the `url` variable with the `next` page URL from the response data.
7. The loop continues until there are no more pages to fetch.
8. Finally, we return the complete list of characters.



Valerie's Note on how to generalize the function in approach 1
we could EASILY generalize this function to say fetch_records: something like this:
***NOTE*** this is untested:

modify the function so that it can be used with any API by passing a base URL and an endpoint as parameters. 
Here's the updated code:

```python
import requests

def fetch_all_records(base_url, endpoint):
    all_records = []

    url = f"{base_url}/{endpoint}"  # Create the complete URL
    
    while url:
        response = requests.get(url)
        data = response.json()
        
        # Change this line to match the endpoint's key in the response data
        all_records.extend(data[f'all_{endpoint}'])  

        url = data['info']['next']  # Get URL of next page

    return all_records

base_url = "https://rickandmortyapi.com/api"
endpoint = "character"
all_characters = fetch_all_records(base_url, endpoint)

print(f"Total {endpoint}s: {len(all_characters)}")
```

Explanation of changes:

1. I've renamed the function to `fetch_all_records` to make it more general and descriptive.
2. The function now takes two parameters: `base_url` and `endpoint`. These parameters will allow you to specify the base URL of any API and the specific endpoint you want to fetch.
3. Inside the function, I've constructed the complete URL by combining the `base_url` and `endpoint`.
4. The line that extends the `all_records` list has been modified to use `data[f'all_{endpoint}']`. This will extract the list of records using a dynamic key based on the endpoint provided. For example, if the endpoint is "character", it will use `data['all_character']`.
5. The rest of the code remains the same.

Now you can use this function to fetch data from any API endpoint by providing the appropriate `base_url` and `endpoint` parameters. Just make sure that the response structure matches the assumed structure (i.e., having a `"results"` field and an `"info"` field with a `"next"` field).



Approach 1 generalized & Extended to include other endpoints

base_url = "https://rickandmortyapi.com/api"
endpoint = "location"
all_locations = fetch_all_records(base_url, endpoint)

print(f"Total {endpoint}s: {len(all_locations)}")


base_url = "https://rickandmortyapi.com/api"
endpoint = "episodes"
all_episodes = fetch_all_records(base_url, endpoint)

print(f"Total {endpoint}s: {len(all_episodes)}")

In [3]:
# **Approach 2: Using Recursive Function**
# This approach uses a recursive function to achieve the same goal, making the code slightly more compact.

import requests

def fetch_characters_recursive(url, all_characters=[]):
    response = requests.get(url)
    data = response.json()
    all_characters.extend(data['results'])
    
    if data['info']['next']:
        fetch_characters_recursive(data['info']['next'], all_characters)

    return all_characters

base_url = "https://rickandmortyapi.com/api/character"
all_characters = fetch_characters_recursive(base_url)

print(f"Total characters: {len(all_characters)}")

Total characters: 826


In [ ]:
Explanation Approach 2:
1. Similar to the previous approach, we define a function `fetch_characters_recursive` that takes the URL and an optional parameter `all_characters` to store the characters.
2. We send a GET request to the API endpoint and extend the `all_characters` list with the characters from the current page.
3. If there is a `next` page URL available, we call the function recursively with the `next` page URL.
4. This recursion continues until there are no more pages to fetch.
5. Finally, we return the complete list of characters.


## Valerie's thoughts on why the `next` key is useful and what happens on the last page of results
When the `"next"` field in the API response is `null`, it means that there are no more pages of data to fetch. This signals the end of the pagination, and both of the provided examples will stop making further API requests. Let's break down what happens in each approach when the `"next"` field is `null`.

**Approach 1: Using a While Loop**

In the first approach, when the `"next"` field becomes `null`, the `while` loop's condition `while url:` will evaluate to `False`, and the loop will exit. This means that the loop stops iterating, and the function `fetch_characters` returns the complete list of characters that have been fetched from all the paginated pages.

**Approach 2: Using Recursive Function**

In the second approach, when the `"next"` field is `null`, the recursive function `fetch_characters_recursive` will not make any further calls to itself. The condition `if data['info']['next']:` checks whether the `"next"` field exists in the response data. If it does not exist or is `null`, the condition will evaluate to `False`, and the recursion will stop. As a result, the function will return the list of characters collected from all the paginated pages.

Both approaches are designed to handle the scenario when there are no more pages to fetch. They ensure that the code execution stops once all available data has been retrieved from the API. This prevents unnecessary API requests and ensures that you have all the data you need for further analysis or processing.

Both approaches achieve the same result: they retrieve paginated data from the API and store it in a list. The first approach uses a `while` loop, while the second approach utilizes recursion. You can choose the one that feels more intuitive to you.

In [2]:
# **Approach 3: using page number as a range**
# If the API provides a `"pages"` field that indicates the total number of pages available, you can use that information to iterate a specific number of times and fetch all the data. 
# This can help optimize your code by making the exact number of required API requests.
# 
import requests

def fetch_characters_with_pages(url):
    all_characters = []

    response = requests.get(url)
    data = response.json()
    all_characters.extend(data['results'])

    total_pages = data['info']['pages']
    
    for page_num in range(2, total_pages + 1):
        response = requests.get(f"{url}?page={page_num}")
        data = response.json()
        all_characters.extend(data['results'])

    return all_characters

base_url = "https://rickandmortyapi.com/api/character"
all_characters = fetch_characters_with_pages(base_url)

print(f"Total characters: {len(all_characters)}")

Total characters: 826


Explanation on Approach 3:
1. We start by sending an initial API request to the base URL to retrieve the data from the first page.
2. We extract the `"pages"` value from the response data, which indicates the total number of pages.
3. We then use a `for` loop to iterate from page number 2 to the total number of pages. (see next cell for further breakdown of how this happens)
4. Inside the loop, we send API requests to the URL with the `page` parameter to fetch each page's data.
5. We extend the `all_characters` list with the characters from each page's response data.
6. The loop continues until all pages have been fetched.
7. Finally, we return the complete list of characters.

By utilizing the `"pages"` field, we can iterate through the exact number of pages without making unnecessary API requests. This approach is efficient and ensures that you retrieve all the available data while minimizing the number of requests.

let's break down the purpose of the `2` and `+1` in the `range` function within the `for` loop.

In Python, the `range` function generates a sequence of numbers, starting from the first argument up to (but not including) the second argument. The `range` function can also take a third argument, which specifies the step size between the numbers. Here's the general syntax of the `range` function:

```python
range(start, stop, step)
```

In the context of the code I provided:

```python
for page_num in range(2, total_pages + 1):
```

- `2` is the start value for the `range`. We start from page number 2 because we've already fetched data from the first page outside the loop. The loop iterates from page number 2 onwards to the total number of pages.
- `total_pages + 1` is the stop value for the `range`. Since the `range` function generates numbers up to, but not including, the stop value, we add `1` to include the total number of pages. For example, if `total_pages` is `42`, this ensures that the loop iterates through pages 2 to 42 (inclusive).

So, the `for` loop iterates over the range of page numbers, fetching data for each page and extending the `all_characters` list with the characters from each page. The purpose of the `2` and `+1` is to ensure that we iterate over the correct range of page numbers to fetch data for all available pages while excluding the first page that we already fetched outside the loop.